In [1]:
import gym
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import numpy as np

In [2]:

def initiate_NN(learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

def epsilon_action(epsilon):
    random_value = np.random.random()
    if random_value<=epsilon:
        action = env.action_space.sample()
    else:
        Y=online_model.predict(state.reshape(1,-1))
        action = np.argmax(Y[0])
    return action


In [3]:


gamma=0.99
epsilon_decay = 0.005
learning_rate =0.0002
max_e=1
min_e=0.0001
replay_batch_size=30
iterations=5000

In [4]:
env = gym.make('LunarLander-v2')
state=env.reset()
online_model = initiate_NN(learning_rate)
target_model =initiate_NN(learning_rate)
target_model.set_weights(online_model.get_weights())
memory=[]
episode_reward_rolling = []
episode_reward=[]

for episode in range(iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                target_q = target_model.predict(np.vstack(batch[:, 3]))
                y[non_terminal_samples] += np.multiply(gamma, \
                            target_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)
        if done:
            target_model.set_weights(online_model.get_weights())

    episode_reward.append(current_episode_reward) 
    if len(memory)>=5000:
        memory = memory[-5000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  0  episode reward:  -182.16  rolling reward:  -182.16  Steps:  108  epsilon:  1.00
episode:  1  episode reward:  -94.46  rolling reward:  -138.31  Steps:  77  epsilon:  1.00
episode:  2  episode reward:  -67.14  rolling reward:  -114.59  Steps:  75  epsilon:  0.99
episode:  3  episode reward:  -150.82  rolling reward:  -123.64  Steps:  77  epsilon:  0.99
episode:  4  episode reward:  -135.38  rolling reward:  -125.99  Steps:  110  epsilon:  0.98
episode:  5  episode reward:  -141.67  rolling reward:  -128.60  Steps:  78  epsilon:  0.98
episode:  6  episode reward:  -206.85  rolling reward:  -139.78  Steps:  77  epsilon:  0.97
episode:  7  episode reward:  -138.88  rolling reward:  -139.67  Steps:  96  epsilon:  0.97
episode:  8  episode reward:  -135.44  rolling reward:  -139.20  Steps:  73  epsilon:  0.96
episode:  9  episode reward:  -194.85  rolling reward:  -144.76  Steps:  115  epsilon:  0.96
episode:  10  episode reward:  -118.35  rolling reward:  -142.36  Steps:  81  e

episode:  89  episode reward:  -97.19  rolling reward:  -132.30  Steps:  90  epsilon:  0.64
episode:  90  episode reward:  -68.89  rolling reward:  -131.60  Steps:  75  epsilon:  0.64
episode:  91  episode reward:  -56.03  rolling reward:  -130.78  Steps:  103  epsilon:  0.63
episode:  92  episode reward:  6.77  rolling reward:  -129.30  Steps:  106  epsilon:  0.63
episode:  93  episode reward:  -63.24  rolling reward:  -128.60  Steps:  151  epsilon:  0.63
episode:  94  episode reward:  -122.93  rolling reward:  -128.54  Steps:  139  epsilon:  0.63
episode:  95  episode reward:  -73.55  rolling reward:  -127.96  Steps:  99  epsilon:  0.62
episode:  96  episode reward:  -77.38  rolling reward:  -127.44  Steps:  104  epsilon:  0.62
episode:  97  episode reward:  -107.21  rolling reward:  -127.24  Steps:  141  epsilon:  0.62
episode:  98  episode reward:  2.89  rolling reward:  -125.92  Steps:  95  epsilon:  0.61
episode:  99  episode reward:  -155.49  rolling reward:  -126.22  Steps:  10

episode:  178  episode reward:  0.92  rolling reward:  -41.00  Steps:  158  epsilon:  0.41
episode:  179  episode reward:  79.21  rolling reward:  -39.15  Steps:  1000  epsilon:  0.41
episode:  180  episode reward:  -205.59  rolling reward:  -40.30  Steps:  355  epsilon:  0.41
episode:  181  episode reward:  29.04  rolling reward:  -39.09  Steps:  1000  epsilon:  0.40
episode:  182  episode reward:  72.03  rolling reward:  -37.01  Steps:  1000  epsilon:  0.40
episode:  183  episode reward:  -14.66  rolling reward:  -36.35  Steps:  128  epsilon:  0.40
episode:  184  episode reward:  185.90  rolling reward:  -33.65  Steps:  966  epsilon:  0.40
episode:  185  episode reward:  -19.81  rolling reward:  -33.21  Steps:  129  epsilon:  0.40
episode:  186  episode reward:  68.07  rolling reward:  -32.32  Steps:  1000  epsilon:  0.39
episode:  187  episode reward:  102.04  rolling reward:  -30.81  Steps:  1000  epsilon:  0.39
episode:  188  episode reward:  7.47  rolling reward:  -30.42  Steps: 

episode:  267  episode reward:  -91.41  rolling reward:  25.38  Steps:  1000  epsilon:  0.26
episode:  268  episode reward:  -21.63  rolling reward:  24.38  Steps:  168  epsilon:  0.26
episode:  269  episode reward:  35.32  rolling reward:  24.57  Steps:  1000  epsilon:  0.26
episode:  270  episode reward:  -92.44  rolling reward:  23.75  Steps:  65  epsilon:  0.26
episode:  271  episode reward:  104.00  rolling reward:  27.29  Steps:  1000  epsilon:  0.26
episode:  272  episode reward:  32.98  rolling reward:  28.18  Steps:  284  epsilon:  0.26
episode:  273  episode reward:  74.33  rolling reward:  28.39  Steps:  1000  epsilon:  0.26
episode:  274  episode reward:  63.28  rolling reward:  28.55  Steps:  1000  epsilon:  0.25
episode:  275  episode reward:  -15.14  rolling reward:  27.85  Steps:  139  epsilon:  0.25
episode:  276  episode reward:  79.98  rolling reward:  27.57  Steps:  1000  epsilon:  0.25
episode:  277  episode reward:  -58.96  rolling reward:  26.38  Steps:  1000  ep

episode:  356  episode reward:  301.48  rolling reward:  78.62  Steps:  453  epsilon:  0.17
episode:  357  episode reward:  238.86  rolling reward:  79.89  Steps:  501  epsilon:  0.17
episode:  358  episode reward:  -46.13  rolling reward:  78.32  Steps:  1000  epsilon:  0.17
episode:  359  episode reward:  233.55  rolling reward:  78.46  Steps:  274  epsilon:  0.17
episode:  360  episode reward:  -214.01  rolling reward:  75.98  Steps:  693  epsilon:  0.17
episode:  361  episode reward:  154.09  rolling reward:  77.78  Steps:  978  epsilon:  0.16
episode:  362  episode reward:  -70.54  rolling reward:  76.56  Steps:  96  epsilon:  0.16
episode:  363  episode reward:  243.49  rolling reward:  78.38  Steps:  306  epsilon:  0.16
episode:  364  episode reward:  234.66  rolling reward:  81.79  Steps:  335  epsilon:  0.16
episode:  365  episode reward:  -99.53  rolling reward:  80.54  Steps:  147  epsilon:  0.16
episode:  366  episode reward:  -81.33  rolling reward:  79.41  Steps:  1000  e

episode:  445  episode reward:  56.14  rolling reward:  132.93  Steps:  187  epsilon:  0.11
episode:  446  episode reward:  235.61  rolling reward:  133.59  Steps:  245  epsilon:  0.11
episode:  447  episode reward:  298.66  rolling reward:  135.64  Steps:  361  epsilon:  0.11
episode:  448  episode reward:  262.90  rolling reward:  139.36  Steps:  352  epsilon:  0.11
episode:  449  episode reward:  -386.43  rolling reward:  132.95  Steps:  178  epsilon:  0.11
episode:  450  episode reward:  25.94  rolling reward:  131.96  Steps:  177  epsilon:  0.11
episode:  451  episode reward:  -8.09  rolling reward:  129.65  Steps:  160  epsilon:  0.10
episode:  452  episode reward:  277.20  rolling reward:  131.41  Steps:  434  epsilon:  0.10
episode:  453  episode reward:  266.66  rolling reward:  132.46  Steps:  867  epsilon:  0.10
episode:  454  episode reward:  278.53  rolling reward:  132.85  Steps:  359  epsilon:  0.10
episode:  455  episode reward:  287.86  rolling reward:  132.87  Steps: 

episode:  534  episode reward:  251.17  rolling reward:  193.99  Steps:  576  epsilon:  0.07
episode:  535  episode reward:  271.50  rolling reward:  197.01  Steps:  227  epsilon:  0.07
episode:  536  episode reward:  277.84  rolling reward:  199.96  Steps:  245  epsilon:  0.07
episode:  537  episode reward:  284.07  rolling reward:  200.46  Steps:  274  epsilon:  0.07
episode:  538  episode reward:  272.20  rolling reward:  201.03  Steps:  193  epsilon:  0.07
episode:  539  episode reward:  280.97  rolling reward:  201.60  Steps:  274  epsilon:  0.07
episode:  540  episode reward:  255.49  rolling reward:  202.07  Steps:  249  epsilon:  0.07
episode:  541  episode reward:  256.84  rolling reward:  204.40  Steps:  254  epsilon:  0.07
episode:  542  episode reward:  237.05  rolling reward:  206.40  Steps:  196  epsilon:  0.07
episode:  543  episode reward:  283.03  rolling reward:  208.83  Steps:  261  epsilon:  0.07
episode:  544  episode reward:  249.04  rolling reward:  208.96  Steps

KeyboardInterrupt: 

In [5]:
online_model.save_weights('weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.h5')

import pickle

with open('weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', 'wb') as fp:
    pickle.dump({'rewards':episode_reward,'rolling_reward':episode_reward_rolling}, fp)
